In [2]:
%pip install mediapipe opencv-python pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# First step

In [4]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as pose:  # Setup mediapipe instance
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )
        
        cv2.imshow("Raw Webcam Feed", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

2024-02-13 22:47:19.371 python[15978:349467] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Save Video

In [1]:
cap = cv2.VideoCapture(0)

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
video_writer = cv2.VideoWriter('deadlift.avi', cv2.VideoWriter_fourcc('P', 'I', 'M', '1'), fps, (int(width), int(height)))

while cap.isOpened():
    ret, frame = cap.read()
    
    cv2.imshow('Press', frame)
    video_writer.write(frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    

cap.release()
# video_writer.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

# Capture Landmarks & Export to CSV

In [7]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [7]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [28]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [29]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints.astype(object), 0, action)
        
        with open('coords.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [31]:
cap = cv2.VideoCapture('benchpress.mp4')
count = 0
with mp_pose.Pose(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as pose:  # Setup mediapipe instance
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )
        
        k = cv2.waitKey(1)
        if k == 119:
            export_landmark(results, 'up')
        if k == 115:
            export_landmark(results, 'down')
        
        cv2.imshow("Raw Webcam Feed", image)
        count += 1

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

# Train Custom Model Using Scikit Learn

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [33]:
df = pd.read_csv('coords.csv')

In [34]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,down,0.400498,0.419187,-0.071906,0.999941,0.386171,0.413787,-0.049498,0.999905,0.385845,...,-0.066293,0.765687,0.804880,1.029398,0.208382,0.251152,0.818988,1.122636,-0.058718,0.691329
1,up,0.402263,0.420550,-0.039944,0.999962,0.387833,0.417942,-0.019979,0.999941,0.387240,...,-0.051767,0.781852,0.839816,1.016859,0.163064,0.263054,0.841883,1.125711,-0.033048,0.699409
2,up,0.391951,0.423795,-0.020689,0.999961,0.379638,0.420832,0.001790,0.999938,0.379679,...,-0.097765,0.801911,0.830809,1.036692,0.149595,0.257014,0.844277,1.129351,-0.083720,0.726549
3,up,0.391103,0.427886,-0.018901,0.999967,0.378304,0.425721,0.003232,0.999947,0.378812,...,-0.097699,0.809279,0.828797,1.042489,0.138024,0.272839,0.843134,1.128589,-0.074921,0.738994
4,up,0.398831,0.427938,-0.017454,0.999978,0.389333,0.419746,0.011327,0.999966,0.390895,...,-0.066092,0.811610,0.809142,1.038660,0.171710,0.313975,0.833190,1.102754,-0.060484,0.758640


In [35]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
36,down,0.404412,0.394142,-0.066365,0.999909,0.391120,0.388576,-0.043276,0.999853,0.391122,...,-0.066074,0.854622,0.798807,1.021753,0.234878,0.310611,0.829087,1.115009,-0.052645,0.817908
37,down,0.406213,0.391614,-0.036895,0.999939,0.392498,0.387116,-0.016067,0.999905,0.392289,...,-0.074010,0.858039,0.813751,1.015423,0.224822,0.302028,0.840089,1.113814,-0.064481,0.809784
38,up,0.386660,0.398293,-0.025243,0.999955,0.374997,0.395276,-0.002466,0.999930,0.375895,...,-0.063221,0.855553,0.814380,1.041757,0.184024,0.316325,0.836775,1.121747,-0.047374,0.810374
39,up,0.394658,0.405179,-0.020502,0.999964,0.382324,0.401446,0.003403,0.999944,0.383676,...,-0.053731,0.851176,0.816017,1.031537,0.173400,0.330286,0.839783,1.114171,-0.032524,0.811067
40,up,0.394312,0.412795,-0.000608,0.999975,0.389275,0.401247,0.027657,0.999960,0.391019,...,-0.021066,0.830130,0.811710,1.005945,0.205676,0.338384,0.841185,1.100135,-0.008020,0.796148


In [36]:
df[df['class'] == 'up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1,up,0.402263,0.420550,-0.039944,0.999962,0.387833,0.417942,-0.019979,0.999941,0.387240,...,-0.051767,0.781852,0.839816,1.016859,0.163064,0.263054,0.841883,1.125711,-0.033048,0.699409
2,up,0.391951,0.423795,-0.020689,0.999961,0.379638,0.420832,0.001790,0.999938,0.379679,...,-0.097765,0.801911,0.830809,1.036692,0.149595,0.257014,0.844277,1.129351,-0.083720,0.726549
3,up,0.391103,0.427886,-0.018901,0.999967,0.378304,0.425721,0.003232,0.999947,0.378812,...,-0.097699,0.809279,0.828797,1.042489,0.138024,0.272839,0.843134,1.128589,-0.074921,0.738994
4,up,0.398831,0.427938,-0.017454,0.999978,0.389333,0.419746,0.011327,0.999966,0.390895,...,-0.066092,0.811610,0.809142,1.038660,0.171710,0.313975,0.833190,1.102754,-0.060484,0.758640
5,up,0.394779,0.429263,0.005701,0.999981,0.388237,0.419128,0.035276,0.999970,0.389829,...,-0.025384,0.809192,0.809616,1.004340,0.195446,0.337047,0.833838,1.095553,-0.016146,0.769815
6,up,0.389942,0.430074,-0.018421,0.999959,0.377523,0.421388,0.011278,0.999933,0.377481,...,-0.027801,0.804753,0.803014,0.999516,0.223024,0.327879,0.834129,1.089398,-0.010304,0.776407
7,up,0.398053,0.443970,-0.000655,0.999886,0.384641,0.443363,0.023006,0.999821,0.384736,...,-0.102850,0.774378,0.800329,1.047682,0.166293,0.279132,0.816036,1.113569,-0.087505,0.756347
13,up,0.396356,0.411170,-0.055267,0.999965,0.385715,0.407502,-0.030935,0.999950,0.386370,...,-0.077201,0.830789,0.804969,1.017237,0.190654,0.346941,0.837007,1.113864,-0.059883,0.798406
14,up,0.396385,0.414505,-0.035649,0.999975,0.389960,0.408336,-0.003639,0.999964,0.392154,...,-0.011146,0.830616,0.807372,0.989005,0.174790,0.370244,0.834006,1.085171,0.004993,0.802347
15,up,0.394825,0.415171,-0.016893,0.999975,0.386940,0.405696,0.009658,0.999960,0.388196,...,-0.032481,0.827868,0.803232,1.025466,0.232112,0.391451,0.826267,1.080934,-0.024367,0.815734


In [37]:
x = df.drop('class', axis=1)
y = df['class']

In [38]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,0.400498,0.419187,-0.071906,0.999941,0.386171,0.413787,-0.049498,0.999905,0.385845,0.413121,...,-0.066293,0.765687,0.804880,1.029398,0.208382,0.251152,0.818988,1.122636,-0.058718,0.691329
1,0.402263,0.420550,-0.039944,0.999962,0.387833,0.417942,-0.019979,0.999941,0.387240,0.417966,...,-0.051767,0.781852,0.839816,1.016859,0.163064,0.263054,0.841883,1.125711,-0.033048,0.699409
2,0.391951,0.423795,-0.020689,0.999961,0.379638,0.420832,0.001790,0.999938,0.379679,0.420717,...,-0.097765,0.801911,0.830809,1.036692,0.149595,0.257014,0.844277,1.129351,-0.083720,0.726549
3,0.391103,0.427886,-0.018901,0.999967,0.378304,0.425721,0.003232,0.999947,0.378812,0.425410,...,-0.097699,0.809279,0.828797,1.042489,0.138024,0.272839,0.843134,1.128589,-0.074921,0.738994
4,0.398831,0.427938,-0.017454,0.999978,0.389333,0.419746,0.011327,0.999966,0.390895,0.418787,...,-0.066092,0.811610,0.809142,1.038660,0.171710,0.313975,0.833190,1.102754,-0.060484,0.758640
5,0.394779,0.429263,0.005701,0.999981,0.388237,0.419128,0.035276,0.999970,0.389829,0.418201,...,-0.025384,0.809192,0.809616,1.004340,0.195446,0.337047,0.833838,1.095553,-0.016146,0.769815
6,0.389942,0.430074,-0.018421,0.999959,0.377523,0.421388,0.011278,0.999933,0.377481,0.421728,...,-0.027801,0.804753,0.803014,0.999516,0.223024,0.327879,0.834129,1.089398,-0.010304,0.776407
7,0.398053,0.443970,-0.000655,0.999886,0.384641,0.443363,0.023006,0.999821,0.384736,0.445320,...,-0.102850,0.774378,0.800329,1.047682,0.166293,0.279132,0.816036,1.113569,-0.087505,0.756347
8,0.405423,0.446384,-0.051613,0.999671,0.393893,0.439359,-0.024164,0.999483,0.394675,0.439701,...,-0.145910,0.737732,0.841458,0.959134,0.147907,0.193616,0.824057,1.082829,-0.149959,0.715283
9,0.404158,0.442902,-0.019818,0.999766,0.393865,0.435572,0.012096,0.999634,0.394980,0.435628,...,-0.094989,0.745600,0.863936,1.003383,0.132599,0.212548,0.840753,1.093651,-0.094066,0.730406


In [39]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [40]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [41]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [42]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [43]:
fit_models['rc'].predict(x_test)

array(['down', 'up', 'down', 'down', 'up', 'down', 'up', 'up', 'up',
       'down', 'down', 'down', 'up'], dtype='<U4')

# Evaluate and Serialize Model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [45]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test.values, yhat),
          precision_score(y_test.values, yhat, average="binary", pos_label="up"),
          recall_score(y_test.values, yhat, average="binary", pos_label="up"))

lr 0.9230769230769231 0.875 1.0
rc 0.9230769230769231 1.0 0.8571428571428571
rf 1.0 1.0 1.0
gb 0.9230769230769231 1.0 0.8571428571428571


In [46]:
yhat = fit_models['rc'].predict(x_test)

In [47]:
yhat[:10]

array(['down', 'up', 'down', 'down', 'up', 'down', 'up', 'up', 'up',
       'down'], dtype='<U4')

In [48]:
y_test
fit_models['lr'].fit(x)

ValueError: This LogisticRegression estimator requires y to be passed, but the target y is None.

In [49]:
with open('deadlift1.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Make Detections with Model

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
with open('deadlift.pkl', 'rb') as f:
    model = pickle.load(f)

In [12]:
cap = cv2.VideoCapture(0)

counter = 0
current_stage = None

with mp_pose.Pose(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as pose:  # Setup mediapipe instance
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66),
                                   thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230),
                                   thickness=2, circle_radius=2),
        )

        try:
            row = np.array(
                [
                    [res.x, res.y, res.z, res.visibility]
                    for res in results.pose_landmarks.landmark
                ]
            ).flatten()
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] <= .75:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] > .75:
                current_stage = 'up'
                counter += 1

            cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

            cv2.putText(
                image,
                "CLASS",
                (95, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                body_language_class.split(" ")[0],
                (90, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

            cv2.putText(
                image,
                "PROB",
                (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                str(round(
                    body_language_prob[np.argmax(body_language_prob)], 2)),
                (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            
            cv2.putText(
                image,
                "COUNT",
                (180, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                str(round(
                    counter)),
                (175, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

        except Exception as e:
            pass

        cv2.imshow("Raw Webcam Feed", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

2024-02-13 22:51:53.566 python[16447:356744] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


: 

In [8]:
with open('deadlift1.pkl', 'rb') as f:
    model = pickle.load(f)
    
cap = cv2.VideoCapture(0)

counter = 0
current_stage = None

with mp_pose.Pose(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as pose:  # Setup mediapipe instance
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66),
                                   thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230),
                                   thickness=2, circle_radius=2),
        )

        try:
            row = np.array(
                [
                    [res.x, res.y, res.z, res.visibility]
                    for res in results.pose_landmarks.landmark
                ]
            ).flatten()
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] <= .75:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] > .75:
                current_stage = 'up'
                counter += 1

            cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

            cv2.putText(
                image,
                "CLASS",
                (95, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                body_language_class.split(" ")[0],
                (90, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

            cv2.putText(
                image,
                "PROB",
                (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                str(round(
                    body_language_prob[np.argmax(body_language_prob)], 2)),
                (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            
            cv2.putText(
                image,
                "COUNT",
                (180, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                str(round(
                    counter)),
                (175, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

        except Exception as e:
            pass

        cv2.imshow("Raw Webcam Feed", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()